In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.1.2



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#https://openblocklabs-interview-datasets.s3.amazonaws.com/market_depth/ekubo_market_depth_dataset.parquet

In [4]:
!aws s3 cp s3://openblocklabs-interview-datasets/market_depth/ekubo_market_depth_dataset.parquet . --request-payer requester

download: s3://openblocklabs-interview-datasets/market_depth/ekubo_market_depth_dataset.parquet to ./ekubo_market_depth_dataset.parquet


In [5]:
import sagemaker
import subprocess

# Initialize SageMaker session and get the default bucket
sess = sagemaker.Session()
bucket = sess.default_bucket()

# Define the source S3 path and target S3 path
source_s3_path = "s3://openblocklabs-interview-datasets/market_depth/ekubo_market_depth_dataset.parquet"
target_s3_path = f"s3://{bucket}/openblocks/data/ekubo_market_depth_dataset.parquet"

# Construct the AWS CLI command for copying
aws_cli_command = f"aws s3 cp {source_s3_path} {target_s3_path} --request-payer requester"

# Execute the command
subprocess.run(aws_cli_command, shell=True, check=True)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
copy: s3://openblocklabs-interview-datasets/market_depth/ekubo_market_depth_dataset.parquet to s3://sagemaker-us-east-1-587092531128/openblocks/data/ekubo_market_depth_dataset.parquet


CompletedProcess(args='aws s3 cp s3://openblocklabs-interview-datasets/market_depth/ekubo_market_depth_dataset.parquet s3://sagemaker-us-east-1-587092531128/openblocks/data/ekubo_market_depth_dataset.parquet --request-payer requester', returncode=0)

In [6]:
import json
import sparknlp
import numpy as np
import pandas as pd
from sparknlp.base import *
from pyspark.ml import Pipeline
from sparknlp.annotator import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline

In [7]:
# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3," "org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )\
    .getOrCreate()


:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a388daf0-76cb-4cd2-93b4-74bbd6f5cd01;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

In [8]:
%%time

df = spark.read.parquet(f"s3a://{bucket}/openblocks/data/ekubo_market_depth_dataset.parquet")

24/03/12 00:08:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 9.13 ms, sys: 735 µs, total: 9.87 ms
Wall time: 8.34 s


In [9]:
#Print the Schema
df.printSchema()

root
 |-- BLOCK_NUMBER: decimal(38,0) (nullable = true)
 |-- BLOCK_TIMESTAMP: timestamp (nullable = true)
 |-- TX_HASH: string (nullable = true)
 |-- TX_ID: string (nullable = true)
 |-- POOL_ID: string (nullable = true)
 |-- TOKEN0_ADDRESS: string (nullable = true)
 |-- TOKEN1_ADDRESS: string (nullable = true)
 |-- EVENT_NAME: string (nullable = true)
 |-- FROM_ADDRESS: string (nullable = true)
 |-- TO_ADDRESS: string (nullable = true)
 |-- TOKEN0_RAW_AMOUNT: decimal(38,0) (nullable = true)
 |-- TOKEN0_DECIMALS: decimal(2,0) (nullable = true)
 |-- TOKEN0_REAL_AMOUNT: decimal(38,30) (nullable = true)
 |-- TOKEN1_RAW_AMOUNT: decimal(38,0) (nullable = true)
 |-- TOKEN1_DECIMALS: decimal(2,0) (nullable = true)
 |-- TOKEN1_REAL_AMOUNT: decimal(38,30) (nullable = true)
 |-- FEE_TIER: float (nullable = true)
 |-- LIQUIDITY_AMOUNT: decimal(38,0) (nullable = true)
 |-- LOWER_TICK: decimal(38,0) (nullable = true)
 |-- UPPER_TICK: decimal(38,0) (nullable = true)
 |-- SWAP_TICK: decimal(38,0) (nu

In [10]:
%%time
num_rows = df.count()
num_columns = len(df.columns)
print(f"DataFrame shape: ({num_rows}, {num_columns})")

DataFrame shape: (3024741, 22)
CPU times: user 1.93 ms, sys: 4.37 ms, total: 6.3 ms
Wall time: 3.01 s


In [23]:
df.show(10)

+------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+-----------------+---------------+--------------------+-----------------+---------------+--------------------+--------+----------------+----------+----------+---------+------------+
|BLOCK_NUMBER|    BLOCK_TIMESTAMP|             TX_HASH|               TX_ID|             POOL_ID|      TOKEN0_ADDRESS|      TOKEN1_ADDRESS|EVENT_NAME|        FROM_ADDRESS|          TO_ADDRESS|TOKEN0_RAW_AMOUNT|TOKEN0_DECIMALS|  TOKEN0_REAL_AMOUNT|TOKEN1_RAW_AMOUNT|TOKEN1_DECIMALS|  TOKEN1_REAL_AMOUNT|FEE_TIER|LIQUIDITY_AMOUNT|LOWER_TICK|UPPER_TICK|SWAP_TICK|TICK_SPACING|
+------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+-----------------+---------------+--------------------+-----

## Analysis 

Market Depth Analysis:
- Calculate market depth for the Ekubo Protocol for STRK/ETH, STRK/USDC, ETH/USDC, USDC/USDT pools.

Profit and Loss (PnL) Calculations:
- From the event data, calculate the PnL for a set of hypothetical positions of the most profitable LPs. Detail your approach and the outcomes of these calculations.


In [27]:
address_counts = df.groupBy("TOKEN0_ADDRESS").count()
top_10_addresses = address_counts.orderBy(col("count").desc())
top_10_addresses.take(5)

[Row(TOKEN0_ADDRESS='0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', count=1979694),
 Row(TOKEN0_ADDRESS='0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8', count=812921),
 Row(TOKEN0_ADDRESS='0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d', count=232126)]

address:  
- eth: `0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc` 
- USDC: `0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8`
- STRK: `0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d`

In [28]:
address_counts = df.groupBy("TOKEN1_ADDRESS").count()
top_10_addresses = address_counts.orderBy(col("count").desc())
top_10_addresses.take(5)

[Row(TOKEN1_ADDRESS='0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8', count=2051110),
 Row(TOKEN1_ADDRESS='0x068f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8', count=819283),
 Row(TOKEN1_ADDRESS='0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', count=154348)]

In [45]:
eth_address = "0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7"
strk_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"
usdc_address = "0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8"
strk_eth_pool = df.filter(
    ((F.col('TOKEN0_ADDRESS') == strk_address) & (F.col('TOKEN1_ADDRESS') == eth_address)) |
    (F.col('TOKEN0_ADDRESS') == eth_address) & (F.col('TOKEN1_ADDRESS') == strk_address)
)
strk_eth_pool.take(5)

[Row(BLOCK_NUMBER=Decimal('536384'), BLOCK_TIMESTAMP=datetime.datetime(2024, 2, 8, 12, 27, 13), TX_HASH='0x0487144ee9f626bb22c74c1d907408cbd7764ce6c9d82ca26a59193bdd332ed1', TX_ID='5461067_0_536384_41', POOL_ID='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b-0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d-0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7-17014118346046923173168730371588410572-95310', TOKEN0_ADDRESS='0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d', TOKEN1_ADDRESS='0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', EVENT_NAME='Mint', FROM_ADDRESS='0x03f60afe30844f556ac1c674678ac4447840b1c6c26854a2df6a8a3d2c015610', TO_ADDRESS='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b', TOKEN0_RAW_AMOUNT=Decimal('0'), TOKEN0_DECIMALS=Decimal('18'), TOKEN0_REAL_AMOUNT=Decimal('0E-30'), TOKEN1_RAW_AMOUNT=Decimal('-100000000000004000'), TOKEN1_DECIMALS=Decimal('18'), TOKEN1

In [46]:
strk_usdc_pool = df.filter(
    ((F.col('TOKEN0_ADDRESS') == strk_address) & (F.col('TOKEN1_ADDRESS') == usdc_address)) |
    (F.col('TOKEN0_ADDRESS') == usdc_address) & (F.col('TOKEN1_ADDRESS') == strk_address)
)
strk_usdc_pool.take(5)

[Row(BLOCK_NUMBER=Decimal('539735'), BLOCK_TIMESTAMP=datetime.datetime(2024, 2, 11, 11, 21, 18), TX_HASH='0x069558394a8e079b7e376cf95d5bfa80680a217efbddd5c19835f1bcc32db011', TX_ID='5461067_0_539735_53', POOL_ID='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b-0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d-0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8-170141183460469235273462165868118016-1000', TOKEN0_ADDRESS='0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d', TOKEN1_ADDRESS='0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8', EVENT_NAME='Mint', FROM_ADDRESS='0x0263fb94ca5addc2a11f8c551067cd434e76395edaa267ec76f9a67b007e97a8', TO_ADDRESS='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b', TOKEN0_RAW_AMOUNT=Decimal('0'), TOKEN0_DECIMALS=Decimal('18'), TOKEN0_REAL_AMOUNT=Decimal('0E-30'), TOKEN1_RAW_AMOUNT=Decimal('-5000000'), TOKEN1_DECIMALS=Decimal('6'), TOKEN1_REAL_AMOUNT=D

In [47]:
eth_usdc_pool = df.filter(
    ((F.col('TOKEN0_ADDRESS') == eth_address) & (F.col('TOKEN1_ADDRESS') == usdc_address)) |
    (F.col('TOKEN0_ADDRESS') == usdc_address) & (F.col('TOKEN1_ADDRESS') == eth_address)
)
eth_usdc_pool.take(5)

[Row(BLOCK_NUMBER=Decimal('328269'), BLOCK_TIMESTAMP=datetime.datetime(2023, 10, 16, 15, 23, 34), TX_HASH='0x03b06aff66afbc79d2e5c2a48919d4c773ef589e7ead338772bd15b9038ee1dc', TX_ID='5461067_0_328269_137', POOL_ID='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b-0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7-0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8-170141183460469235273462165868118016-1000', TOKEN0_ADDRESS='0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', TOKEN1_ADDRESS='0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8', EVENT_NAME='Mint', FROM_ADDRESS='0x04c7b913ce8d1d7a0bfcdad2546abfdb657958660a069af364969af3216fbaaa', TO_ADDRESS='0x00000005dd3d2f4429af886cd1a3b08289dbcea99a294197e9eb43b0e0325b4b', TOKEN0_RAW_AMOUNT=Decimal('-632340897114350'), TOKEN0_DECIMALS=Decimal('18'), TOKEN0_REAL_AMOUNT=Decimal('-0.000632340897114350044931618822'), TOKEN1_RAW_AMOUNT=Decimal('-997738'), TOKEN1

In [50]:
buy_transactions_count = eth_usdc_pool.filter(F.col('EVENT_NAME') == 'Mint').count()
sell_transactions_count = eth_usdc_pool.filter(F.col('EVENT_NAME') == 'Burn').count()
print(f"buy transactions count in eth/usdc pool: {buy_transactions_count}")
print(f"sell transactions count in eth/usdc pool: {sell_transactions_count}")

buy transactions count in eth/usdc pool: 230199
sell transactions count in eth/usdc pool: 115395


In [51]:
buy_transactions_count = strk_usdc_pool.filter(F.col('EVENT_NAME') == 'Mint').count()
sell_transactions_count = strk_usdc_pool.filter(F.col('EVENT_NAME') == 'Burn').count()
print(f"buy transactions count in strk/usdc pool: {buy_transactions_count}")
print(f"sell transactions count in strk/usdc pool: {sell_transactions_count}")

buy transactions count in strk/usdc pool: 6858
sell transactions count in strk/usdc pool: 1877


In [52]:
buy_transactions_count = strk_eth_pool.filter(F.col('EVENT_NAME') == 'Mint').count()
sell_transactions_count = strk_eth_pool.filter(F.col('EVENT_NAME') == 'Burn').count()
print(f"buy transactions count in strk/eth pool: {buy_transactions_count}")
print(f"sell transactions count in strk/eth pool: {sell_transactions_count}")

buy transactions count in strk/eth pool: 17361
sell transactions count in strk/eth pool: 4482


## Ticks to price in ekubo protocol 

https://docs.ekubo.org/about-ekubo/key-concepts

- Ticks correspond to exact prices which can be used to specify boundaries for positions. Position boundaries are specified as ticks rather than arbitrary prices so the AMM can more efficiently process swaps. Ticks could be defined arbitrarily, but the most uniform way to break up the entire price range is in log-space.

- In Ekubo, the tick size is 1.000001, or 1/100th of a basis point. You can get the price by raising 1.000001 to the power tick. This level of precision is smaller than most centralized limit order books. In other words, that means you can create one-tick positions, similar to limit orders, at prices more specific than you could on centralized exchanges. This makes Ekubo a great place to build an on-chain order book.

In [54]:
def tick_to_price(tick):
    return 1.000001 ** tick


In [56]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def tick_to_price(tick):
    # Convert Decimal tick to float before performing the operation
    return 1.000001 ** float(tick)


# register UDF
tick_to_price_udf = udf(tick_to_price, DoubleType())

# apply UDF
df_with_prices = df.withColumn("price_low", tick_to_price_udf("LOWER_TICK"))\
                   .withColumn("price_high", tick_to_price_udf("UPPER_TICK"))

df_with_prices.select("LOWER_TICK", "UPPER_TICK", "price_low", "price_high").show()


+----------+----------+--------------------+--------------------+
|LOWER_TICK|UPPER_TICK|           price_low|          price_high|
+----------+----------+--------------------+--------------------+
|    -20000|     20000|  0.9801986831103483|  1.0202013298230708|
|    -20000|     20000|  0.9801986831103483|  1.0202013298230708|
| -20284000| -20244000|1.551583435245099...|1.614904724158150...|
| -20279000| -20239000|1.559360775680326...|1.622999463715774...|
|    -20000|     20000|  0.9801986831103483|  1.0202013298230708|
|    -20000|     20000|  0.9801986831103483|  1.0202013298230708|
| -20287000| -20247000|1.546935662408715E-9|1.610067272210807...|
| -20287000| -20247000|1.546935662408715E-9|1.610067272210807...|
| -20275000| -20235000|1.565610707187346...|1.629504459636125E-9|
| -20279000| -20243000|1.559360775680326...|1.616520435795496E-9|
| -29935000| -29934000|9.98622129012372E-14|9.996212501190351...|
| -20281000| -20241000|1.556245172328908...|1.619756710244375...|
| -2027400

In [ ]:
from pyspark.sql.functions import col

# 使用orderBy对DataFrame进行排序
strk_usdc_pool_sorted = strk_usdc_pool.orderBy(col('P_lower'))

# 使用PySpark的窗口函数计算累积流动性
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as _sum

windowSpec = Window.orderBy(col('P_lower')).rowsBetween(Window.unboundedPreceding, Window.currentRow)
strk_usdc_pool_sorted = strk_usdc_pool_sorted.withColumn('Cumulative_Liquidity', _sum('Liquidity').over(windowSpec))

# 显示结果
strk_usdc_pool_sorted.select('P_lower', 'Cumulative_Liquidity').show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# 绘制累积流动性
plt.plot(df_sorted['P_lower'], df_sorted['Cumulative_Liquidity'], label='累积流动性')

plt.xlabel('价格')
plt.ylabel('累积流动性')
plt.title('市场深度图')
plt.legend()
plt.show()